# Pipeline de extracción, este es el bueno

In [40]:
import fitz
import pandas as pd
import nltk
import os 

nltk.download('cess_esp')
from nltk.corpus import cess_esp
#Por mejorar, este corpus está chiquito
spanish_words = set(word for word in cess_esp.words())

path = "data/"
documents = []

# Get all the PDF documents NAMES from the path
for file in os.listdir(path):
    if file.endswith(".pdf"):
        documents.append(file[:-4])


[nltk_data] Downloading package cess_esp to
[nltk_data]     C:\Users\Maracuya\AppData\Roaming\nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


In [41]:
import re


def get_text_from_pdf(key):
    doc = fitz.open(f'data/{key}.pdf')
    number_of_pages = doc.page_count
    full_text=""
    punctuation = '''!()-[]{};:'"\,<>./?_—’'''
    regex = r'[^\w\s' + punctuation + ']' 
    for page in doc.pages():
        this_page_text=page.get_text("text", sort=True).replace("\n", " ")
        #Quitar caracteres especiales
        this_page_text=re.sub(regex, '', this_page_text)
        this_page_text=re.sub(r'\d+', '', this_page_text)
        
        #Quitar más de un underscore, es para temas de completado en libros educativos
        this_page_text = re.sub('_+', ' ', this_page_text)
        #Quitar cualquier espacio o tabulación extra
        this_page_text = re.sub('[ \t]+', ' ', this_page_text)


        this_page_text = this_page_text.encode("ascii", errors="ignore").decode()
        full_text+=this_page_text+" "
    return full_text

def get_all_text_from_pdf(key):
    doc = fitz.open(f'data/{key}.pdf')
    full_text=" ".join([page.get_text("text", sort=True).replace("\n", " ") for page in doc])
    full_text=re.sub(r'\d+', '', full_text)
    return full_text

In [42]:
from nltk.tokenize import sent_tokenize
from langdetect import detect
import langid

def filtrar_espaniol(full_text):
    sentences = sent_tokenize(full_text)
    non_spanish_sentences = []

    
    for sentence in sentences:
        try:
            languages=[]
            languages.append(detect(sentence))
            languages.append(langid.classify(sentence)[0])
            if not any(lang in languages for lang in ["es", "it", "ca", "pt"]):
                non_spanish_sentences.append(sentence)
        except:
            pass

    sentences = non_spanish_sentences
    

    return sentences

def filtrar_oracion_espaniol(sentence):
    try:
        languages=[]
        languages.append(detect(sentence))
        languages.append(langid.classify(sentence)[0])
        if not any(lang in languages for lang in ["es", "it", "ca", "pt"]):
            return True
    except:
        pass

    return False

In [43]:
#Funciones de filtrado
alfabeto_quechua = ['a', 'aa', 'ch', 'chh', 'ch\'', 'ts', 'tr', 'h', 'i', 'ii', 'k', 'kh', 'k\'', 'l', 'll', 'm', 'n', 'ñ', 'p', 'ph', 'p\'', 'q', 'qh', 'q\'', 'r', 's', 'sh', 't', 'th', 't\'', 'u', 'uu', 'w', 'y']

def grafemas_no_en_alfabet(words):
    for word in words:
        for i, letter in enumerate(word):
            #Continue  if letter is not a letter
            if not letter.isalpha():
                continue
            if letter.lower() not in alfabeto_quechua:
                #Chequear siguiente letra
                if i+1 >= len(word):
                    return False
                letter = letter + word[i+1]
                if letter.lower() not in alfabeto_quechua:
                    if i+2 >= len(word):
                        return False
                    #Chequear siguiente letra
                    letter = letter + word[i+2]
                    if letter.lower() not in alfabeto_quechua:
                        return False
    return True

#No funciona bien esta función, porque el detector de lenguaje no es muy bueno para palabras 
def oraciones_mucho_espaniol(words):
    spanish_words = 0
    for word in words:
        try:
            if detect(word) == 'es':
                spanish_words += 1
        except:
            pass
    return spanish_words/len(words) > 0.5

def oracion_mucho_espaniol_v2(words):
    palabras_encontradas = 0
    #Usar un diccionario de palabras en español
    for word in words:
        if word.lower() in spanish_words:
            palabras_encontradas += 1

    return palabras_encontradas/len(words) < 0.25


def oraciones_muy_cortas(words, min_length=3):
    return len(words) > min_length

def oraciones_muy_repititivas(words, threshold=0.5):
    #Todo a minúsculas
    words = [word.lower() for word in words]
    
    unique_words = set(words)
    ratio = len(unique_words) / len(words)
    return ratio >= threshold

def palabras_muy_largas(words, threshold=40):
    for word in words:
        if len(word) > threshold:
            return False
    return True

def split_tokens(sentence):
    #Esta expresión busca cualquier secuencia de 2 o más palabras que tengan cada una uno o dos caracteres de longitud, separadas por espacios.
    #En el paper se indica que es para detectar partes de una palabra que se rompieron durante la extracción de texto debido al formato del pdf.
    if re.search(r"(\b\w{1,2}\b\s){2,}", sentence):
        return False
    return True

def oraciones_con_matematica(sentence):
    if re.search(r"[\d+\-*/]+", sentence):
        return False
    return True

In [44]:
#a, aa, ch, chh, ch', ts, tr, h, i, ii, k, kh, k', l, ll, m, n, ñ, p, ph, p', q, qh, q', r, s, sh, t, th, t', u, uu, w, y
from nltk.tokenize import word_tokenize
# Reglas basadas en https://aclanthology.org/2020.lrec-1.356/
def rule_based_heuristic(sentence):
    words = word_tokenize(sentence)
    aux_solo_palabras = [word for word in words if word.isalpha()]
    
    valid = oraciones_muy_cortas(aux_solo_palabras) 
    valid = valid and oraciones_muy_repititivas(aux_solo_palabras)
    valid = valid and palabras_muy_largas(words)
    valid = valid and split_tokens(sentence)
    valid = valid and oraciones_con_matematica(sentence)
    valid = valid and oracion_mucho_espaniol_v2(aux_solo_palabras)

    return valid

In [45]:
def pipeline(documents):
    corpus=pd.DataFrame(columns=["document", "sentence"])
    for key in documents:
        text=get_text_from_pdf(key)
        sentences=sent_tokenize(text)
        #Oraciones antes de filtrar
        print("Oraciones antes de filtrar en documento ", key, ": ", len(sentences))
        filtered_sentences=[]
        for sentence in sentences:
            if rule_based_heuristic(sentence) and filtrar_oracion_espaniol(sentence):
                filtered_sentences.append(sentence)
        
        df=pd.DataFrame(filtered_sentences, columns=["sentence"])
        df["document"]=key
        #Quitar duplicados
        df.drop_duplicates(subset="sentence", inplace=True)
        print("Oraciones después de filtrar en documento ", key, ": ", len(df))
        corpus=pd.concat([corpus, df], ignore_index=True)
    print("Total de oraciones: ", len(corpus))
    corpus.drop_duplicates(subset="sentence", inplace=True)
    print("Total de oraciones únicas: ", len(corpus))
    return corpus

In [46]:
df_corpus=pipeline(documents)
df_corpus.head()

Oraciones antes de filtrar en documento  1 Rimana. Qichwa - Qullawpi llamk'ana mayt'u. Cuaderno de trabajo - Comunicación 1° - Quechua Collao :  2291
Oraciones después de filtrar en documento  1 Rimana. Qichwa - Qullawpi llamk'ana mayt'u. Cuaderno de trabajo - Comunicación 1° - Quechua Collao :  1362
Oraciones antes de filtrar en documento  3 Rimana - Qillqasqa Mayt’u Qichwa Qullaw. Texto de Comunicación del 3° Secundaria - Quechua Collao :  1334
Oraciones después de filtrar en documento  3 Rimana - Qillqasqa Mayt’u Qichwa Qullaw. Texto de Comunicación del 3° Secundaria - Quechua Collao :  727
Oraciones antes de filtrar en documento  4 Rimana - Qillqasqa Mayt’u Qichwa Qullaw. Texto de Comunicación del 4° Secundaria - Quechua Collao :  1430
Oraciones después de filtrar en documento  4 Rimana - Qillqasqa Mayt’u Qichwa Qullaw. Texto de Comunicación del 4° Secundaria - Quechua Collao :  782
Oraciones antes de filtrar en documento  5 Rimana - Qillqasqa Mayt’u Qichwa Qullaw. Texto de Comunic

,document,sentence
0,1 Rimana. Qichwa - Qullawpi llamk'ana mayt'u. ...,Llamk'ana qillqana mayt'upin tarinki imaymana ...
1,1 Rimana. Qichwa - Qullawpi llamk'ana mayt'u. ...,"Llamk'anaykipaqtaq yachachiqniyki, tayta mamay..."
2,1 Rimana. Qichwa - Qullawpi llamk'ana mayt'u. ...,Watuchkaykuan qampaq allin kayninta hinallataq...
3,1 Rimana. Qichwa - Qullawpi llamk'ana mayt'u. ...,Kay llamk'ana qillqana mayt'uqa k}humpaykin ya...
4,1 Rimana. Qichwa - Qullawpi llamk'ana mayt'u. ...,"Qillqayta, awinchayta kusisqalla yachay!"


In [62]:
#Random sample
for sentence in df_corpus.sample(10)['sentence'].values:
    print(sentence)

Imataq mikhuy waqaychayri Mikhuy waqaychayqa awpa runap kawsayninmantaraqmi.
Khaynatan siqikurani kimsa watachayuq kachkaspay, kaytataq tawa watachayuq kaspay.
Aleli nin: ataq kay wayqachay thantachaa chayqa, llakisqan karqani mana maypi llamkanachaykunata waqaychayta atispa.
Huk punchawsi, uwihanta michichkaspa sipaschaqa liw, liw, liw nispa liqichup waqasqanta uyarisqa.
Qillqap wachunkunata allinta awinchaspa, hamitasqaykimanhina qillqna maytuykipi qillqay.
Chay pukllanapaqqa huk kachu runa akllakun.
Imaynatataq Peru suyupiri khuchita uywanku?
Manchana ayllu runakuna, imatam quriwan achalakuqku?
Mana riqsisqa rimaykunata, llapan qillqata qhawarispa chuyanchanki.
Sapa tuqupin utaq muhuta churasqaku, tukurquspaataqsi wakap wanunwan qatasqanku.


In [66]:
df_corpus.to_csv("data/corpus/corpus.csv", index=False)